## FINM33150 Market Flow and Trading Opportunity
#### Eric Chu

In [259]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from datetime import timedelta, date
from datetime import datetime
import threading

### Import data
###### Trade data

In [260]:
# Trade data
trades_BTC_USD = pd.read_csv('trades_narrow_BTC-USD_2021.delim.gz', compression ='gzip', sep ='\t')
print('BTC/USD Trades Control Total: %d' % len(trades_BTC_USD))

trades_ETH_BTC = pd.read_csv('trades_narrow_ETH-BTC_2021.delim.gz', compression ='gzip', sep ='\t')
print('ETH/BTC Trades Control Total: %d' % len(trades_ETH_BTC))

trades_ETH_USD = pd.read_csv('trades_narrow_ETH-USD_2021.delim.gz', compression ='gzip', sep ='\t')
print('ETH/USD Trades Control Total: %d' % len(trades_ETH_USD))

BTC/USD Trades Control Total: 6268838
ETH/BTC Trades Control Total: 32744
ETH/USD Trades Control Total: 306798


###### Book Data

In [261]:
# # Book data
# book_BTC_USD = pd.read_csv('book_narrow_BTC-USD_2021.delim.gz', compression ='gzip', sep ='\t')
# print('BTC/USD Book Control Total: %d' % len(book_BTC_USD))

# book_ETH_BTC = pd.read_csv('book_narrow_ETH-BTC_2021.delim.gz', compression ='gzip', sep ='\t')
# print('ETH/BTC Book Control Total: %d' % len(book_ETH_BTC))

# book_ETH_USD = pd.read_csv('book_narrow_ETH-USD_2021.delim.gz', compression ='gzip', sep ='\t')
# print('ETH/USD Book Control Total: %d' % len(book_ETH_USD))

BTC/USD Book Control Total: 15000000
ETH/BTC Book Control Total: 2569095
ETH/USD Book Control Total: 11802439


###### Index data on timestamp

In [262]:
# Trade data
trades_BTC_USD.set_index('timestamp_utc_nanoseconds', inplace = True)
trades_ETH_BTC.set_index('timestamp_utc_nanoseconds', inplace = True)
trades_ETH_USD.set_index('timestamp_utc_nanoseconds', inplace = True)

# Book data
book_BTC_USD.set_index('timestamp_utc_nanoseconds', inplace = True)
book_ETH_BTC.set_index('timestamp_utc_nanoseconds', inplace = True)
book_ETH_USD.set_index('timestamp_utc_nanoseconds', inplace = True)

###### Clean and prep the data a bit
Size = this is the directional size of the trade using the side column  
Notional Size - this is the price multiplied by the size. This will be used to calculate VWAP as an average price for each nanosecond  
Data is aggregated from tick to the nanosecond level

In [268]:
# Data cleaning
def data_clean(df):
    df['Size'] = df['SizeBillionths'] * np.where(df['Side'] > 0, 1, -1)
    df['notional size'] = df['PriceMillionths'] * df['SizeBillionths']
    # Create VWAP column for the price at each tick
    df_out = df.groupby(df.index).agg({'Size':'sum',
                                                'SizeBillionths':'sum',
                                                'notional size':'sum'})
    df_out['VWAP'] = df_out['notional size'] / df_out['SizeBillionths']
    return df_out

In [269]:
trades_BTC_USD_VWAP = data_clean(trades_BTC_USD)
print('BTC/USD ns level Control Total: %d' % len(trades_BTC_USD_VWAP))
trades_ETH_BTC_VWAP = data_clean(trades_ETH_BTC)
print('ETH/BTC ns level Control Total: %d' % len(trades_ETH_BTC_VWAP))
trades_ETH_USD_VWAP = data_clean(trades_ETH_USD)
print('ETH/USD ns level Control Total: %d' % len(trades_ETH_USD_VWAP))

BTC/USD ns level Control Total: 4896190
ETH/BTC ns level Control Total: 31399
ETH/USD ns level Control Total: 260713


### Split data into test and train

In [270]:
###### Split datasets into train and test sets
def split_data(df, percent = .2):
    end_i = int(round(len(df) * .20, 0))
    train = df.iloc[:end_i]
    test = df.iloc[end_i:]
    return train, test

In [271]:
train_BTC_USD, test_BTC_USD = split_data(trades_BTC_USD_VWAP)
print('BTC/USD Train Control Total: %d' % len(train_BTC_USD))
print('BTC/USD Test Control Total: %d\n' % len(test_BTC_USD))

train_ETH_BTC, test_ETH_BTC = split_data(trades_ETH_BTC_VWAP)
print('ETH/BTC Train Control Total: %d' % len(train_ETH_BTC))
print('ETH/BTC Test Control Total: %d\n' % len(test_ETH_BTC))

train_ETH_USD, test_ETH_USD = split_data(trades_ETH_USD_VWAP)
print('ETH/USD Train Control Total: %d' % len(train_ETH_USD))
print('ETH/USD Test Control Total: %d\n' % len(test_ETH_USD))

BTC/USD Train Control Total: 979238
BTC/USD Test Control Total: 3916952

ETH/BTC Train Control Total: 6280
ETH/BTC Test Control Total: 25119

ETH/USD Train Control Total: 52143
ETH/USD Test Control Total: 208570



### Calculate T interval trade flow prior to each trade data point

We use the following formula to calculate trade flow
$$ F_{t}^{(T)} = V_{(t-\tau,t)}^B - V_{(t-\tau,t)}^S $$

In [279]:
# create a function to calculate T-window tradeflow, 
def tradeflow(df, t):
    tau = t * (10**9)
    x = datetime.now()
    print('Start: ' + str(x))
    # consolidate trade sizes by time
    dfout = df.copy()
    
    # get start and end parameters for each row in the dataframe
    dfout['start'] = dfout.index - tau
    dfout['end'] = dfout.index - 1
    
    # list to append flow values to
    calcs = []
    
    # locks for threading
    flag = threading.Lock()
    flag2 = threading.Lock()
    
    # divide threading job up
    n = 6
    interval = len(dfout) // n

    # function to get flow values
    def ops (dfin, start, stop):
        flag.acquire()
        df_temp = dfin.copy()
        flag.release()
        temp_list = []
        for i in range(start,stop):
            index_val = df_temp.index[i]
            start = df_temp.loc[index_val,'start']
            end = df_temp.loc[index_val,'end']
            flow = df_temp.loc[start:end]['Size'].sum()
            temp_list.append((index_val,flow))
        
        flag2.acquire()
        nonlocal calcs
        calcs += temp_list
        flag2.release()
        
    # multithread the above procedures
    threads = []
    
    for i in range(n):
        s = i*interval
        e = (i+1) * interval
        if i == (n-1):
            e = len(dfout)
        t = threading.Thread(target = ops, args = (dfout,s,e))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()
    
    # append results to dataframe
    calcs_pd = pd.DataFrame(calcs)
    calcs_pd.set_index(0,inplace = True)
    calcs_pd.rename(columns = {1:'tradeflow'}, inplace = True)
    dfout = pd.concat([dfout,calcs_pd], axis = 1)

        
    # cut off the beginning of data equal to tau since those values of Trading Flow are not properly calculated
    cutoff = dfout.index[0] + tau
    dfout = dfout.loc[cutoff:]
    
    dfout.drop(columns = ['start','end'], axis = 1, inplace = True)
    dfout.index = pd.to_datetime(dfout.index)
    print(datetime.now()-x)
    
    return dfout

In [280]:
# train_BTC_USD_1s = tradeflow(train_BTC_USD,1)
train_ETH_BTC_1s = tradeflow(train_ETH_BTC,1)
train_ETH_USD_1s = tradeflow(train_ETH_USD,1)

# train_BTC_USD_5s = tradeflow(train_BTC_USD,5)
train_ETH_BTC_5s = tradeflow(train_ETH_BTC,5)
train_ETH_USD_5s = tradeflow(train_ETH_USD,5)

# train_BTC_USD_10s = tradeflow(train_BTC_USD,10)
train_ETH_BTC_10s = tradeflow(train_ETH_BTC,10)
train_ETH_USD_10s = tradeflow(train_ETH_USD,10)

# print('BTC/USD 1s Control Total: %d' % len(train_BTC_USD_1s))
print('ETH/BTC 1s Control Total: %d' % len(train_ETH_BTC_1s))
print('ETH/USD 1s Control Total: %d' % len(train_ETH_USD_1s))

# print('BTC/USD 5s Control Total: %d' % len(train_BTC_USD_5s))
print('ETH/BTC 5s Control Total: %d' % len(train_ETH_BTC_5s))
print('ETH/USD 5s Control Total: %d' % len(train_ETH_USD_5s))

# print('BTC/USD 10s Control Total: %d' % len(train_BTC_USD_10s))
print('ETH/BTC 10s Control Total: %d' % len(train_ETH_BTC_10s))
print('ETH/USD 10s Control Total: %d' % len(train_ETH_USD_10s))

Start: 2022-02-07 04:02:26.013178
0:00:01.584548
Start: 2022-02-07 04:02:27.598751
0:00:12.776558
Start: 2022-02-07 04:02:40.384272
0:00:01.530625
Start: 2022-02-07 04:02:41.914897
0:00:12.657024
Start: 2022-02-07 04:02:54.580897
0:00:01.524551
Start: 2022-02-07 04:02:56.106446
0:00:12.703690
BTC/USD 1s Control Total: 1005463
ETH/BTC 1s Control Total: 6278
ETH/USD 1s Control Total: 52140
BTC/USD 5s Control Total: 0
ETH/BTC 5s Control Total: 6278
ETH/USD 5s Control Total: 52132
BTC/USD 10s Control Total: 0
ETH/BTC 10s Control Total: 6277
ETH/USD 10s Control Total: 52123


###### Calculate returns at time T

In [281]:
# function that takes a dataframe and returns a column with the time T returns
def time_T_ret(df,t):
    tau = t * (10**9)
    later = df['VWAP'].reindex(df.index + pd.Timedelta(tau, unit='ns'), method = 'ffill')
    later.index = df.index
    ret2 = later / df['VWAP'] - 1
    returns_2_day = pd.DataFrame({'Time-T Return':ret2})
    out = pd.concat([df,returns_2_day], axis=1)
    return out

In [282]:
# train_BTC_USD_1s_full = time_T_ret(train_BTC_USD_1s,19)
train_ETH_BTC_1s_full = time_T_ret(train_ETH_BTC_1s,1)
train_ETH_USD_1s_full = time_T_ret(train_ETH_USD_1s,1)

# train_BTC_USD_5s_full = time_T_ret(train_BTC_USD_5s,5)
train_ETH_BTC_5s_full = time_T_ret(train_ETH_BTC_5s,5)
train_ETH_USD_5s_full = time_T_ret(train_ETH_USD_5s,5)

# train_BTC_USD_10s_full = time_T_ret(train_BTC_USD_10s,10)
train_ETH_BTC_10s_full = time_T_ret(train_ETH_BTC_10s,10)
train_ETH_USD_10s_full = time_T_ret(train_ETH_USD_10s,10)

###### Perform regression of the returns on the tradeflow and get the betas to use on the test sets

In [284]:
def regression(df):
    x = sm.add_constant(df['tradeflow'])
    y = df['Time-T Return']

    # good ole OLS
    model = sm.OLS(y,x)

    reg = model.fit()

#     print(reg.summary())
    
    return reg.params[1]

In [285]:
# BTC_USD_1s_beta = regression(train_BTC_USD_1s_full)
ETH_BTC_1s_beta = regression(train_ETH_BTC_1s_full)
ETH_USD_1s_beta = regression(train_ETH_USD_1s_full)

# BTC_USD_5s_beta = regression(train_BTC_USD_5s_full)
ETH_BTC_5s_beta = regression(train_ETH_BTC_5s_full)
ETH_USD_5s_beta = regression(train_ETH_USD_5s_full)

# BTC_USD_10s_beta = regression(train_BTC_USD_10s_full)
ETH_BTC_10s_beta = regression(train_ETH_BTC_10s_full)
ETH_USD_10s_beta = regression(train_ETH_USD_10s_full)

###### Use betas on test sets

In [288]:
def trade(df,beta,t,j, label):
    j = j
    apply_beta = tradeflow(df,t)
    get_ret = time_T_ret(apply_beta,t)
    get_ret['beta'] = beta
    get_ret['predicted'] = get_ret['beta'] * get_ret['tradeflow']

    def position(x):
        nonlocal j
        if x > j:
            return 1
        elif x < -j:
            return -1
        else:
            return 0
    
    get_ret['trade_position'] = get_ret['predicted'].apply(position)

    get_ret['trade_return'] = get_ret['Time-T Return'] * get_ret['trade_position']
    
    shorts = len(get_ret[get_ret['trade_position'] == -1])
    longs = len(get_ret[get_ret['trade_position'] == 1])
    total = shorts + longs
    print('*****************************\n' + label + '\n*****************************')
    print("Long positions: %d\nShort positions: %d\nTotal Positions: %d" % (longs,shorts,total))

    print('Predicted Trade EV: %.6f\n\n' % get_ret['trade_return'].sum())

In [290]:
# trade(test_BTC_USD,BTC_USD_1s_beta,1,.000025, 'BTC_USD 1s Stragety')
trade(test_ETH_BTC,ETH_BTC_1s_beta,1,.000025, 'ETH_BTC 1s Stragety')
trade(test_ETH_USD,ETH_USD_1s_beta,1,.000025, 'ETH_USD 1s Stragety')

# trade(test_BTC_USD,BTC_USD_5s_beta,5,.000025, 'BTC_USD 5s Stragety')
trade(test_ETH_BTC,ETH_BTC_5s_beta,5,.000025, 'ETH_BTC 5s Stragety')
trade(test_ETH_USD,ETH_USD_5s_beta,5,.000025, 'ETH_USD 5s Stragety')

# trade(test_BTC_USD,BTC_USD_10s_beta,10,.000025, 'BTC_USD 10s Stragety')
trade(test_ETH_BTC,ETH_BTC_10s_beta,10,.000025, 'ETH_BTC 10s Stragety')
trade(test_ETH_USD,ETH_USD_10s_beta,10,.000025, 'ETH_USD 10s Stragety')

Start: 2022-02-07 04:06:55.613912
0:00:08.700068
*****************************
ETH_BTC 1s Stragety
*****************************
Long positions: 122
Short positions: 155
Total Positions: 277
Predicted Trade EV: 0.005749


Start: 2022-02-07 04:07:04.372823
0:00:52.126431
*****************************
ETH_USD 1s Stragety
*****************************
Long positions: 48463
Short positions: 102885
Total Positions: 151348
Predicted Trade EV: -9641.600761


Start: 2022-02-07 04:07:56.782070
0:00:06.114111
*****************************
ETH_BTC 5s Stragety
*****************************
Long positions: 0
Short positions: 1
Total Positions: 1
Predicted Trade EV: 0.000000


Start: 2022-02-07 04:08:02.933064
0:00:50.771878
*****************************
ETH_USD 5s Stragety
*****************************
Long positions: 69066
Short positions: 137264
Total Positions: 206330
Predicted Trade EV: -9184.271855


Start: 2022-02-07 04:08:53.986141
0:00:06.077917
*****************************
ETH_BTC 10s Str